# Projet final

### Acquisition et chargement des données

* Récupération des fichiers Excel avec les classements
* Mise en place d'une copie locale des fichiers Excel afin de ne pas les recharger à chaque run.
* Vers la fin de la course le format des fichiers Excel change avec les arrivées des voiliers : il est possible de s'arrêter juste avant.



In [19]:
# Chargement des libs
import pandas as pd
import os
import sys
from bs4 import BeautifulSoup as bs
import requests

# libs for xlsx "xxid" fix
import tempfile
from zipfile import ZipFile
import shutil
from fnmatch import fnmatch
import re
import glob
#

# variables utiliséees globalement
URL_RESULTS="https://www.vendeeglobe.org/fr/classement"
EXCELS_DIR="results"
PICKLE_DF="vendee_globe.pkl"

### Question 1

* Récupération des fichiers Excel avec les classements
* Mise en place d'une copie locale des fichiers Excel afin de ne pas les recharger à chaque run.
* Chargement dans un dataframe (clean up et split des noms de colonne, préparation des data pour traitements)

In [143]:
def get_soup_from_url(url):
    """
    Retourne la soupe de l'url du fichier html passé en paramètre
    """
    res = requests.get(url)
    soup = bs(res.content, 'html.parser')
    return soup

def get_soup_from_file(file):
    """
    Retourne la soupe du fichier html passé en paramètre
    """
    soup = bs(file, 'html.parser')
    return soup

def parse_url_for_excels(url):
    """
    Récupère la liste des fichiers excel et les télécharge dans le répertoire "results/" (EXCELS_DIR)
    """
    print("Getting url of files to download...")
    soup = get_soup_from_url(url)
    dates_list = []
    for option in soup.find_all('option'):
        if option['value'] != '':
            dates_list.append(option['value'])
    
    # format de fichiers à récupérer 
    # https://www.vendeeglobe.org/download-race-data/vendeeglobe_20210305_080000.xlsx
    print("Downloading xlsx files...")
    for date in dates_list:
        xlsx_name = f'vendeeglobe_{date}.xlsx'
        xlsx_file = requests.get(f"https://www.vendeeglobe.org//download-race-data/{xlsx_name}")
        open(os.path.join(EXCELS_DIR, xlsx_name), 'wb').write(xlsx_file.content)

def fix_xlsx_errors():
    """
    Fix des fichiers xlsx, un header xxid dans un des fichiers du xlsx n'est pas reconnu par openpyxl
    """
    print("Fixing xlsx files...")
    for file in [f for f in glob.glob(EXCELS_DIR + "/*.xlsx")]:
        change_in_zip(file, name_filter='xl/styles.xml', # the problematic property is found in the style xml files
                      change=lambda d: re.sub(b'xxid="\d*"', b"", d))
        
# fix of xlsx files
def change_in_zip(file_name, name_filter, change):
    """
    le fix appliqué à chaque fichier
    """
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, 'new.zip')
        with ZipFile(file_name, 'r') as r, ZipFile(tempname, 'w') as w:
            for item in r.infolist():
                data = r.read(item.filename)          
                data = change(data)
                w.writestr(item, data)
        shutil.move(tempname, file_name)
    finally:
        shutil.rmtree(tempdir)


def get_excel_files():
    if not os.path.isdir(EXCELS_DIR):
        os.mkdir('results')
        print("Files being downloaded to ", EXCELS_DIR)
        parse_url_for_excels(URL_RESULTS)
        fix_xlsx_errors()
    else:
        print(f"Files are already in directory \"{EXCELS_DIR}\" and processed, no files downloaded, no files processed")


**L'appel à la commande *get_excel_files.xlsx* ne fait rien si le répertoire *results/* existe. Il est crée lors du 1er téléchargement** 

In [6]:
get_excel_files()

Files are already in directory "results" and processed, no files downloaded, no files processed


In [31]:
#content = pd.ExcelFile('vendeeglobe_20210305_080000.xlsx', engine="openpyxl") 
#print(content)

In [1]:
# def open_openpyxl(path, **kw):
#     import openpyxl
#     default_kw = {'read_only': True, 'data_only': True, 'keep_links': False}
#     for k,v in default_kw.items():
#         if k not in kw:
#             kw[k] = v
#         wb = openpyxl.load_workbook(path, **kw)
#         sheet = wb.worksheets[0]
#     print(sheet.calculate_dimension())
#     from pandas.io.excel._openpyxl import OpenpyxlReader
#     convert_cell = OpenpyxlReader(path)._convert_cell
#     data = []
#     for row in sheet.rows:
#         data.append([convert_cell(cell, False) for cell in row])
#     return data

# path='xxid.xlsx'
# b = open_openpyxl(path, read_only=False)


In [3]:
# from openpyxl import load_workbook
# wb = load_workbook(filename = xlsxpath)
# wb

Index(['Classement', 'Pays', 'Skipper', 'Heure FR', 'Latitude', 'Longitude',
       'Cap 30m', 'Vitesse 30m', 'VMG 30m', 'Distance 30m', 'Cap dernier',
       'Vitesse dernier', 'VMG dernier', 'Distance dernier', 'Cap 24h',
       'Vitesse 24h', 'VMG 24h', 'Distance 24h', 'DTF', 'DTL',
       'Fichier de resultats', 'Voile', 'Bateau', 'REMOVE'],
      dtype='object')

In [103]:
x.head(1)
ddd = create_dataframe_from_files("small")
ddd = clean_data(ddd)
def clean2(df):
    for col in ['Vitesse 30m','VMG 30m', 'Vitesse dernier', 'VMG dernier', 'Vitesse 24h', 'VMG 24h']:
#         print(col)
        df[[col]] = df[col].str.extract("(.*) kts").astype(float)
    for col in ['Distance 30m', 'Distance dernier', 'Distance 24h', 'DTF', 'DTL']:
        df[[col]] = df[col].str.extract("(.*) nm").astype(float)
    for col in ['Cap 30m', 'Cap dernier', 'Cap 24h']:
        df[[col]] = df[col].str.extract("(.*) nm").astype(float)

    df[['Classement']] = df[['Classement']].applymap(lambda x: x.replace('RET', '0')).astype(int)
    return df
# ddd = clean2(ddd)
# ddd.info()

Including file  small/vendeeglobe_20210101_170000.xlsx
Cleaning dataframe... Vitesse 30m
VMG 30m
Vitesse dernier
VMG dernier
Vitesse 24h
VMG 24h
done


In [104]:
def create_dataframe_from_files(path, verbose=True):
    """
    Sélection des fichiers et concatenate dans une dataframe
    """
    if not verbose:
        print("Quiet mode activated. Be patient...", end='')
    dfs=[]
    for filename in [f for f in glob.glob(path + "/*.xlsx")]:

        # on exclu les fichiers à partir du premier arrivé
        # on exclu le 1er fichier au départ qui est sans données
        if filename >= path+'/vendeeglobe_20210127_170000.xlsx' or filename == path+"/vendeeglobe_20201108_120200.xlsx":
            continue
        if verbose:
            print('Including file ', filename)
        # do not use col 0
        # remove footer
        # rename 1  Classement
        # split 2 on \n rename Pays /  Voile
        # split 3 on \n ren Skipper /  Bateau
        # change names
        
        # read excel
        x = pd.read_excel(filename, 
                          dtype=object,
                          skiprows=[1, 2, 3], 
                          header=1, 
                          usecols=range(1,21), 
                          skipfooter=4)
        
        # ajout colonne avec le timestamp du fichier d'où est extrait la data
        x['Fichier de resultats'] = filename[-20:-5] # yobAAAA.txt
        
        dfs.append(x)
    
    df = pd.concat(dfs , ignore_index=True)
    if not verbose:
        print("done")
    return df

def clean_data(df, verbose=True):
    """
    Cleanup par
    """
    print('Cleaning dataframe... ', end='')

    # cleanup sur nom dde cols, etc.
    df.rename(inplace=True, columns={'Unnamed: 1': 'Classement', 'Unnamed: 2': 'Pays', 'Unnamed: 3': 'Skipper', 
                                    'Unnamed: 19': 'DTF', 'Unnamed: 20': 'DTL', "Heure FR\nHour FR": "Heure FR"})
    df[['Pays','Voile']] = df["Pays"].str.extract("(.*)\n(.*)").astype(str)
    df[['Skipper','Bateau']] = df["Skipper"].str.extract("(.*)\n(.*)").astype(str)
    df[['Heure FR','REMOVE']] = df["Heure FR"].str.extract("(.*)\n(.*)").astype(str)
    df.rename(inplace=True, columns={'Latitude\nLatitude': 'Latitude', 
                    'Longitude\nLongitude': 'Longitude', 
                    'Cap\nHeading': 'Cap 30m', 'Vitesse\nSpeed': 'Vitesse 30m', 
                    'VMG\nVMG': 'VMG 30m', 'Distance\nDistance': 'Distance 30m',
                    'Cap\nHeading.1': 'Cap dernier', 'Vitesse\nSpeed.1': 'Vitesse dernier', 
                    'VMG\nVMG.1': 'VMG dernier', 'Distance\nDistance.1': 'Distance dernier',
                    'Cap\nHeading.2': 'Cap 24h', 'Vitesse\nSpeed.2': 'Vitesse 24h', 
                    'VMG\nVMG.2': 'VMG 24h', 'Distance\nDistance.2': 'Distance 24h'})

    for col in ['Vitesse 30m','VMG 30m', 'Vitesse dernier', 'VMG dernier', 'Vitesse 24h', 'VMG 24h']:
        print(col)
        df[[col]] = df[col].str.extract("(.*) kts").astype(float)
    for col in ['Distance 30m', 'Distance dernier', 'Distance 24h', 'DTF', 'DTL']:
        df[[col]] = df[col].str.extract("(.*) nm").astype(float)
    for col in ['Cap 30m', 'Cap dernier', 'Cap 24h']:
        df[[col]] = df[col].str.extract("(.*) nm").astype(float)

    df[['Classement']] = df[['Classement']].applymap(lambda x: x.replace('RET', '-1')).astype(str)
    df[['Classement']] = df[['Classement']].applymap(lambda x: x.replace('NL', '-2')).astype(str)
    
    print("done")
    return df



if not os.path.isfile(PICKLE_DF):
    print(f"\nLoading data from {EXCELS_DIR}/*.xlsx files")
    df = create_dataframe_from_files(EXCELS_DIR, verbose=False)
    df = clean_data(df)
    df.to_pickle(PICKLE_DF)
else:
    print(f"\nLoading data from pickle file...", end='')
    df = pd.read_pickle(PICKLE_DF)
    print(' done')
    print(f'\n(Note: if you need to renew pickle content remove file {PICKLE_DF} manually)')
    
df.shape


Loading data from pickle file... done

(Note: if you need to renew pickle content remove file vendee_globe.pkl manually)


(15906, 24)

In [249]:
df.set_index('Skipper')


,Classement,Pays,Heure FR,Latitude,Longitude,Cap 30m,Vitesse 30m,VMG 30m,Distance 30m,Cap dernier,...,VMG 24h,Distance 24h,DTF,DTL,Fichier de resultats,Voile,Bateau,REMOVE,foil,foil2
Skipper,,,,,,,,,,,,,,,,,,,,,
Charlie Dalin,1,,08:30 FR,32°21.07'S,18°09.12'W,NaN,5.9,5.9,2.9,NaN,...,12.0,289.4,19215.0,0.0,20201124_080000,FRA 79,APIVIA,,NaN,NaN
Thomas Ruyant,2,,08:30 FR,31°20.72'S,18°13.22'W,NaN,10.2,10.2,5.1,NaN,...,11.1,268.9,19251.9,36.9,20201124_080000,FRA 59,LinkedOut,,NaN,NaN
Jean Le Cam,3,,08:30 FR,28°01.14'S,21°13.47'W,NaN,7.7,7.4,3.8,NaN,...,10.4,256.3,19496.2,281.2,20201124_080000,FRA 01,Yes we Cam !,,NaN,NaN
Kevin Escoffier,4,,08:30 FR,27°45.23'S,22°14.46'W,NaN,10.7,10.6,5.3,NaN,...,9.4,225.4,19548.7,333.7,20201124_080000,FRA 85,PRB,,NaN,NaN
Yannick Bestaven,5,,08:30 FR,27°06.17'S,23°39.08'W,NaN,12.4,12.4,6.2,NaN,...,9.2,224.9,19632.0,417.0,20201124_080000,FRA 17,Maître Coq IV,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Samantha Davies,-1,,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20201227_170000,FRA 109,Initiatives - Coeur,nan,NaN,NaN
Sébastien Simon,-1,,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20201227_170000,FRA 4,ARKEA PAPREC,nan,NaN,NaN
Alex Thomson,-1,,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20201227_170000,GBR 99,HUGO BOSS,nan,NaN,NaN


In [303]:
def get_infos_from_classement(df):
    """
    Récupère les infos skipper, foil, etc.
    """
    print("Getting boats infos...")
    if not os.path.isfile("classement.html"):
        print("Reading file from far away")
        classement_html = requests.get(URL_RESULTS)
        open(os.path.join("classement.html"), 'wb').write(classement_html.content)
    else:
        print("Reading file locally")
        with open('classement.html','r') as file:
            classement_html = file.read()

    soup = get_soup_from_file(classement_html)
    skippers_info=[]
    for ranking_row in soup.find_all("tr", {"class": "ranking-row"}):
        cell_rank = ranking_row.find('td', attrs={'class': 'row-number'} ).text
        cell_skipper = ranking_row.find('td', attrs={'class': 'row-skipper'} ).contents[2]
        cell_skipper = re.search(r'\n\s+(\w[\s\'\w-]*)', cell_skipper).group(1).title()
        cell_has_foil = ranking_row.find('td', attrs={'class': 'row-layout'} ).text
        skippers_info.append([cell_rank, cell_skipper, cell_has_foil])
    return skippers_info


foils_etc = get_infos_from_classement(df)

# manual fixes
# skippers with names not matching
# from web page 2 errors
skips=set()
for f in foils_etc:
    skipper=f[1]
    skips.add(skipper)
all_skips = set(df['Skipper'])
print(skips.difference(set(all_skips)))
df[['Skipper']] = df[['Skipper']].applymap(lambda x: x.replace('Arnaud Boissieres', 'Arnaud Boissières')).astype(str)

for f in foils_etc:
    if f[1]=="Sam Davies":
        skipper = "Samantha Davies"
    elif f[1]=="Alan  Roura":
        skipper="Alan Roura"
    else:
        skipper=f[1]
    foil=f[2]
    cl_final=lambda x: int(x) if x!="ABD" else int(-1), f[0]
    cl_final(f[0])
    
    df.loc[df['Skipper']==skipper, 'foil'] = foil
    df.loc[df['Skipper']==skipper, 'Classement final'] = cl_final
df

Getting boats infos...
Reading file locally
{'Alan  Roura', 'Sam Davies'}


TypeError: 'tuple' object is not callable

In [262]:
set(df['Skipper'])

{'Alan Roura',
 'Alex Thomson',
 'Alexia Barrier',
 'Ari Huusela',
 'Armel Tripon',
 'Arnaud Boissières',
 'Benjamin Dutreux',
 'Boris Herrmann',
 'Charlie Dalin',
 'Clarisse Cremer',
 'Clément Giraud',
 'Damien Seguin',
 'Didac Costa',
 'Fabrice Amedeo',
 'Giancarlo Pedote',
 'Isabelle Joschke',
 'Jean Le Cam',
 'Jérémie Beyou',
 'Kevin Escoffier',
 'Kojiro Shiraishi',
 'Louis Burton',
 'Manuel Cousin',
 'Maxime Sorel',
 'Miranda Merron',
 'Nicolas Troussel',
 'Pip Hare',
 'Romain Attanasio',
 'Samantha Davies',
 'Stéphane Le Diraison',
 'Sébastien Destremau',
 'Sébastien Simon',
 'Thomas Ruyant',
 'Yannick Bestaven'}

In [33]:
xlsxpath='results/vendeeglobe_20201110_080000.xlsx'
x = pd.read_excel(xlsxpath, skiprows=[1, 2, 3], 
                          header=1, 
                          usecols=range(1,21), 
                          skipfooter=4)
        
# ajout colonne avec le timestamp du fichier d'où est extrait la data
x['Fichier de resultats'] = xlsxpath[-20:-5]
x = clean_data(x)
x

Cleaning dataframe done


,Classement,Pays,Skipper,Heure FR,Latitude,Longitude,Cap 30m,Vitesse 30m,VMG 30m,Distance 30m,...,Cap 24h,Vitesse 24h,VMG 24h,Distance 24h,DTF,DTL,Fichier de resultats,Voile,Bateau,REMOVE
0,1,,Jérémie Beyou,09:30 FR,43°28.72'N,10°31.39'W,270°,9.5 kts,3.3 kts,4.8 nm,...,202°,9.2 kts,9.2 kts,221.7 nm,23921.1 nm,0.0 nm,20201110_080000,FRA 8,Charal,
1,2,,Jean Le Cam,09:30 FR,43°10.90'N,09°20.59'W,231°,8.2 kts,7.2 kts,4.1 nm,...,212°,7.8 kts,7.7 kts,186.3 nm,23923.3 nm,2.2 nm,20201110_080000,FRA 01,Yes we Cam !,
2,3,,Damien Seguin,09:30 FR,43°11.81'N,09°14.41'W,246°,3.2 kts,2.3 kts,1.6 nm,...,209°,7.8 kts,7.7 kts,186.3 nm,23925.8 nm,4.8 nm,20201110_080000,FRA 1000,Groupe APICIL,
3,4,,Benjamin Dutreux,09:30 FR,43°12.68'N,09°15.17'W,240°,8.1 kts,6.4 kts,4.1 nm,...,205°,8.4 kts,8.4 kts,201.5 nm,23926.4 nm,5.4 nm,20201110_080000,FRA 09,OMIA - Water Family,
4,5,,Nicolas Troussel,09:30 FR,43°13.67'N,09°16.68'W,234°,4.4 kts,3.8 kts,2.2 nm,...,187°,9.0 kts,8.7 kts,211.0 nm,23926.9 nm,5.9 nm,20201110_080000,FRA 6,CORUM L'Épargne,
5,6,,Maxime Sorel,09:30 FR,43°14.58'N,09°17.24'W,226°,6.8 kts,6.3 kts,3.4 nm,...,209°,7.6 kts,7.6 kts,183.2 nm,23927.6 nm,6.6 nm,20201110_080000,FRA 53,V And B Mayenne,
6,7,,Kevin Escoffier,09:30 FR,43°33.29'N,10°18.26'W,262°,9.6 kts,4.6 kts,4.8 nm,...,202°,9.2 kts,9.2 kts,220.7 nm,23928.7 nm,7.6 nm,20201110_080000,FRA 85,PRB,
7,8,,Charlie Dalin,09:30 FR,43°39.78'N,10°33.60'W,252°,8.4 kts,5.2 kts,4.2 nm,...,205°,9.0 kts,9.0 kts,215.5 nm,23930.9 nm,9.8 nm,20201110_080000,FRA 79,APIVIA,
8,9,,Romain Attanasio,09:30 FR,43°15.11'N,09°06.80'W,229°,8.5 kts,8.4 kts,4.3 nm,...,206°,8.1 kts,8.1 kts,193.6 nm,23931.3 nm,10.2 nm,20201110_080000,FRA 49,Pure - Best Western Hotels and Resorts,
9,10,,Samantha Davies,09:30 FR,43°36.39'N,10°08.22'W,267°,8.0 kts,3.2 kts,4.0 nm,...,203°,8.9 kts,8.9 kts,212.9 nm,23934.2 nm,13.1 nm,20201110_080000,FRA 109,Initiatives - Coeur,


In [28]:
# remove nan lines
# extract pour chaque bateau la vitesse / 24 en histogramme
# correlation vitesse distance
# 


KeyError: "None of [Index(['132°', '119°', '142°', '127°', '139°', '136°', '129°', '143°', '187°',\n       '158°',\n       ...\n       '101°',  '76°',  '61°',  '96°',    nan,    nan,    nan,    nan,    nan,\n          nan],\n      dtype='object', length=15906)] are in the [columns]"


* Extraction des caractéristiques techniques de chacun des voiliers.
* Rapprochement des données des voiliers avec celle des classements.
* Corrélation et régression linéaire entre le classement (rang) et la vitesse utile (VMG) des voiliers.
* Impact de la présence d'un foil sur le classement et la vitesse des voiliers.
* Visualisation de la distance parcourue par voilier.
* Cartes avec les routes d'un ou plusieurs voiliers.
* Analyses de séries temporelles.
* Application d'algorithmes statistiques ou de machine learning.
* etc.